In [40]:
DATA_DIR = "/home/user/newnew/"

In [41]:
!pip install sklearn
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.externals import joblib

import socket, struct

In [42]:
packets_1 = pd.read_csv(DATA_DIR + "1")
packets_2 = pd.read_csv(DATA_DIR + "2")
packets_1['uid'] = 1
packets_2['uid'] = 2
packets = packets_1.append([packets_2])

In [43]:
def ip2int(addr):                                                               
    try:
        return struct.unpack("!I", socket.inet_aton(addr))[0]                       
    except OSError:
        return 0

In [44]:
def encode(x, enc):
    try:
        return enc.transform(x)
    except ValueError:
        return -1

In [45]:
def preprocessing(packets, label_encoder):
    pd.options.mode.chained_assignment = None
    packets["Source"] = packets["Source"].apply(lambda ip : ip2int(ip)).astype(np.int32)
    packets["Destination"] = packets["Destination"].apply(lambda ip : ip2int(ip)).astype(np.int32)
    packets["Protocol"] = packets["Protocol"].apply(lambda x: encode(x, label_encoder))

In [46]:
%%time
le = LabelEncoder().fit(packets["Protocol"])
X = packets[["Source", "Destination", "Protocol", "Length"]]
preprocessing(X, le)
y = packets["uid"]

CPU times: user 704 ms, sys: 16 ms, total: 720 ms
Wall time: 710 ms


In [47]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17)

In [49]:
%%time
forest = RandomForestClassifier(n_jobs=-1, random_state=17)
forest.fit(X_train, y_train)
print(f1_score(y_holdout, forest.predict(X_holdout), average='binary'))

0.998331943286
CPU times: user 56 ms, sys: 12 ms, total: 68 ms
Wall time: 228 ms
